In [1]:
import numpy as np

# 解析 TSP 文件
def parse_tsp(file_name):
    with open(file_name, 'r') as file:
        lines = file.readlines()
        cities = []
        for line in lines:
            parts = line.split()
            if len(parts) == 3 and parts[0].isdigit():
                cities.append((float(parts[1]), float(parts[2])))
        return cities

# 计算两点间距离
def distance(city1, city2):
    return np.sqrt((city1[0] - city2[0])**2 + (city1[1] - city2[1])**2)

# 计算路径长度
def path_length(cities, tour):
    total_distance = 0
    for i in range(len(tour)):
        total_distance += distance(cities[tour[i-1]], cities[tour[i]])
    return total_distance

# 2-opt 算法
def two_opt_swap(tour, i, j):
    new_tour = list(tour[:i])
    new_tour.extend(reversed(tour[i:j + 1]))
    new_tour.extend(tour[j + 1:])
    return np.array(new_tour)


# 解析 .opt.tour 文件，以比较最优路径长度
def parse_opt_tour(file_name):
    with open(file_name, 'r') as file:
        lines = file.readlines()
        opt_tour = []
        for line in lines:
            parts = line.split()
            if parts[0].isdigit():
                opt_tour.append(int(parts[0]) - 1)  # 减1是因为TSP文件通常从1开始编号，而Python列表从0开始
        return opt_tour

# 根据 .opt.tour 文件计算路径长度
def calculate_opt_length(cities, opt_tour):
    total_distance = 0
    for i in range(len(opt_tour)):
        total_distance += distance(cities[opt_tour[i - 1]], cities[opt_tour[i]])
    return total_distance

# 模拟退火算法
def simulated_annealing(cities, initial_temp, final_temp, alpha):
    current_temp = initial_temp
    current_tour = np.random.permutation(len(cities))
    current_length = path_length(cities, current_tour)
    
    while current_temp > final_temp:
        i, j = np.sort(np.random.choice(len(cities), 2, replace=False))
        new_tour = two_opt_swap(current_tour, i, j)
        new_length = path_length(cities, new_tour)
        
        if new_length < current_length or np.random.rand() < np.exp((current_length - new_length) / current_temp):
            current_tour = new_tour
            current_length = new_length
            
        current_temp *= alpha
    
    return current_tour, current_length




In [2]:
# 解析 TSP 和 OPT 文件
cities = parse_tsp('a280.tsp.txt')  # 替换为 a280.tsp 文件的实际路径
opt_tour = parse_opt_tour('a280.opt.tour.txt')  # 替换为 a280.opt.tour 文件的实际路径
opt_length = calculate_opt_length(cities, opt_tour)

In [3]:
# 不同冷却策略
# 修改模拟退火算法以适应不同的冷却计划
def simulated_annealing(cities, initial_temp, final_temp, alpha, cooling_schedule):
    current_temp = initial_temp
    best_tour = np.random.permutation(len(cities))
    best_length = path_length(cities, best_tour)
    iteration = 1

    while current_temp > final_temp:
        for _ in range(len(cities)):
            i, j = np.sort(np.random.choice(len(cities), 2, replace=False))
            new_tour = two_opt_swap(best_tour, i, j)
            new_length = path_length(cities, new_tour)

            if new_length < best_length or np.random.rand() < np.exp((best_length - new_length) / current_temp):
                best_tour = new_tour
                best_length = new_length

        current_temp = cooling_schedule(current_temp, alpha, iteration)
        iteration += 1

    return best_length

# 不同的冷却计划
def linear_cooling(current_temp, alpha):
    return current_temp - alpha

def exponential_cooling(current_temp, alpha):
    return current_temp * alpha

def logarithmic_cooling(current_temp, alpha, iteration):
    return current_temp / (1 + alpha * np.log(1 + iteration))

# 多次运行模拟退火算法
def multiple_runs_simulated_annealing(cities, runs, initial_temp, final_temp, alpha, cooling_schedule):
    best_length = float('inf')
    
    for _ in range(runs):
        length = simulated_annealing(cities, initial_temp, final_temp, alpha, cooling_schedule)
        if length < best_length:
            best_length = length
    
    return best_length

In [5]:
# 设置参数并运行模拟退火算法多次
runs = 1  # 运行次数
initial_temp = 10000  # 初始温度
final_temp = 1        # 最终温度
alpha = 0.997         # 冷却率

# 运行模拟退火算法并比较不同冷却计划
cooling_strategies = {
    #"Linear Cooling": lambda temp, alpha, _: linear_cooling(temp, alpha),
    "Exponential Cooling": lambda temp, alpha, _: exponential_cooling(temp, alpha),
    #"Logarithmic Cooling": logarithmic_cooling
}

results = {}
for name, strategy in cooling_strategies.items():
    best_length = multiple_runs_simulated_annealing(cities, runs, initial_temp, final_temp, alpha, strategy)
    results[name] = best_length
    print(f"{name}: {best_length}")

Exponential Cooling: 2846.779722518119
